In [ ]:
#First we will import packages that contain functions we will use here. 

#Pandas is a great python package for dataframe manipulation.
import pandas as pd

read_table is the command that will read in a tab separated file as a dataframe. See what other pandas read functions are available by pressing tab after pd.read. Auto-complete will show you all the options. 

We are going to set the index of the dataframe as the first column rather than an arbitrary number. (Try this function both with and without setting the index, how is it different??) If you look at the file we are loading with less on the command line, you will see that there are comments at the top of the file on lines that start with #. We need to tell pandas to ignore those when loading the dataframe. So we will use comment = "#".

To make sure that all manipulations are doing what we expect, we will also print the shape of the dataframe (do the number of rows and columns make sense) and look at the beginning of the dataframe with df.head()

In [ ]:
counts = pd.read_table("/oasis/tscc/scratch/biom200/featurecounts/featureCounts.txt", index_col=0,
                      comment="#")
print counts.shape
counts.head()

In [ ]:
#This is the syntax to make a list in python. Lists are surrounded by square brackets. 

#We don't care about a few columns in the dataframe, so let's get rid of them.

cols_to_drop = ['Chr','Start','End','Strand']

#The command to get rid of rows is df.drop
#We provide a list of columns to drop, and the axis that contains these values (1 is columns, 0 is rows)
counts = counts.drop(cols_to_drop, axis=1)
print counts.shape
counts.head()

Notice how dropping 4 columns changed the number of rows in the dataframe. 

The column names are pretty annoying because they list the full path and the bam file name. Let's rename them to something shorter. We can use

    counts.columns
    
to give us a list of column names. This is easy to copy the ones we want into a dictionary that we will make below. 

In [ ]:
counts.columns

A dictionary lets us link a key:value pair. In this instance we are using a key that is the old name and a value that is the new name. We will use this pairing scheme to define all old:new column names and feed that into a function to rename columns. 

Dictionaries are make with {"key":"value", "key2":"value2", "key3":"value3}

In [ ]:
col_names = {'/home/ucsd-train01/projects/fto_shrna/all_bams/k562_FTO_shRNA_rep1_Aligned.out.sorted.bam':"FTO_shrna_rep1",
       '/home/ucsd-train01/projects/fto_shrna/all_bams/k562_FTO_shRNA_rep2_Aligned.out.sorted.bam':"FTO_shrna_rep2",
       '/home/ucsd-train01/projects/fto_shrna/all_bams/k562_NT_shRNA_rep1_Aligned.out.sorted.bam':"FTO_control_rep1",
       '/home/ucsd-train01/projects/fto_shrna/all_bams/k562_NT_shRNA_rep2_Aligned.out.sorted.bam':"FTO_control_rep2"}

You can put a dictionary in the rename function to rename the columns. Let's feed in the dictionary that we made called col_names. Check the shape and head of the dataframe to make sure the changes happened as you expected them to.

In [ ]:
counts = counts.rename(columns = col_names)
print counts.shape
counts.head()

You will notice that this dataframe has almost 58,000 genes. That's a lot. And it looks like there might be a lot with very few counts. So we are going to calculate the mean counts across all of our samples and get rid of genes that have a mean count of less than 5.

You will notice that our dataframe also contains length information

In [ ]:
counts.columns

Let's make a list of the columns that we want to look at when calculating the mean. We don't want to include the Length value in our mean calculation. 

In [ ]:
samples = ['FTO_shrna_rep1', 'FTO_shrna_rep2', 'FTO_control_rep1', 'FTO_control_rep2']

Access only those columns by putting the list in square brackets after the name of the dataframe. Calculate mean across the rows with .mean(axis=1). Let's take a look at the first 5 results .head()

In [ ]:
counts[samples].mean(axis=1).head()

Set the filtering cutoff at 5. We want to keep genes that have a mean count value greater than 5. Use a boolean to find out which genes have a mean greater than 5. This returns a True/False array of genes to keep.

In [ ]:
counts[samples].mean(axis=1) > 5

Let's save this result and call it genes_to_keep. Notice a few key things. The True/False is described by the GeneID. The geneid is also the index of our dataframe. This is very important!!

In [ ]:
genes_to_keep = counts[samples].mean(axis=1) > 5

Since the geneID is the index of our dataframe, we can use .loc to only keep instances where the geneID is True. So the syntax of the following command is dataframe.loc[]. Inside the square bracket is an array describing for each item in the index, whether or not to keep it. True items are kept, false items are removed. Take a look at how this affects how many rows are in the dataframe with .shape. How many genes are left in our analysis? 

In [ ]:
counts_clean = counts.loc[genes_to_keep]
print counts_clean.shape
counts_clean.head()

To find out how many True values were in our array, we can use .sum(). Sum in this case will count the number of Trues. (True has a value of 1, False has a value of 0). genes_to_keep.sum() should match the number of rows in our dataframe. Does it? 

In [ ]:
genes_to_keep.sum()

For DESeq2, we need to provide a counts matrix. We are going to use the filtered counts matrix to get rid of the genes with 0 (or nearly 0) counts. But we don't want the length column. Remember to access only a few rows, give the name of the dataframe followed by square brackets. Inside that square bracket, give it a list of the columns you want to use. We made a list called samples so this is the list we will use to keep only our samples and ignore the Length column.

In [ ]:
counts_clean[samples].head()

Save this dataframe as a csv. First define the directory where you want to save it. Make sure this directory exists (make it on your command line first). Call it deseq_dir. Put it in quotes to tell python this is a string. 

To save, use .to_csv() and put in the name of the file where you want to save it. This is the directory + a meaningful filename. spaces are important here, don't space between the directory and the string with the file name. Follow the syntax exactly as written below.

In [ ]:
deseq_dir = "/home/ucsd-train01/projects/fto_shrna/deseq2/"

counts_clean[samples].to_csv(deseq_dir+"fto_counts_for_deseq2.csv")

DESeq2 also needs a conditions matrix where the row names are the sample names (that exactly match the column names from the counts matrix) and there is one column describing the condition that the sample came from. We can set the row names (index) directly when making a new dataframe by saying index = samples. Remember samples is a list of column names from our counts matrix above. 

In [ ]:
conditions = pd.DataFrame(index = samples)
conditions.head()

To make a new column in a dataframe, put square brackets after the dataframe with the new column name in quotes. In this dataframe, we will make a new column called 'condition'. Set this equal to a list of the values that you would like to fill this column. In our case, the values are knockdown or control depending on the sample. Look at the conditions dataframe with head to make sure it is doing what you think it is. 

In [ ]:
conditions['condition'] = ['knockdown','knockdown','control','control']

In [ ]:
conditions.head()

Save this dataframe the same way we did before.

In [ ]:
conditions.to_csv(deseq_dir+"fto_conditions_for_deseq2.csv")

We also want to save the counts matrix with the Length column (we need this to calculate TPM and FPKM later). Take a look at that dataframe to make sure it is what we want and save it to the featurecounts directory that you have likely already made. 

In [ ]:
counts_clean.head()

In [ ]:
feature_counts_dir = "/home/ucsd-train01/projects/fto_shrna/featurecounts/"

counts_clean.to_csv(feature_counts_dir+"fto_clean_counts_with_length.csv")